<a href="https://colab.research.google.com/github/gazon1/Recco-challenge/blob/master/bmp25_%2B_lightfm_with_features_60epochs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Rekko challenge 2019

```
                           /$$$$$$$  /$$$$$$$$ /$$   /$$ /$$   /$$  /$$$$$$ 
                          | $$__  $$| $$_____/| $$  /$$/| $$  /$$/ /$$__  $$
                          | $$  \ $$| $$      | $$ /$$/ | $$ /$$/ | $$  \ $$
                          | $$$$$$$/| $$$$$   | $$$$$/  | $$$$$/  | $$  | $$
                          | $$__  $$| $$__/   | $$  $$  | $$  $$  | $$  | $$
                          | $$  \ $$| $$      | $$\  $$ | $$\  $$ | $$  | $$
                          | $$  | $$| $$$$$$$$| $$ \  $$| $$ \  $$|  $$$$$$/
                          |__/  |__/|________/|__/  \__/|__/  \__/ \______/ 
                                                                            
```

Добро пожаловать на соревнование по машинному обучению от онлайн-кинотеатра [Okko](http://okko.tv) Rekko Challenge 2019.

В этом ноутбуке мы покажем вам пример простого но полного решения, от загрузки данных до формирования ответа. Для работы нам понадобятся библиотеки `pandas`, `numpy`, `scipy`, `implicit`, `pprint`, `tqdm`. Установить их в вашем рабочем окружении можно следующей командой.
```
pip install pandas numpy scipy implicit pprint tqdm
```

In [0]:
import os
import json
import pandas as pd

import numpy as np


import tqdm
import scipy.sparse as sp
from sklearn.metrics import average_precision_score as score
import time
from matplotlib import pyplot as plt

from sklearn import preprocessing
from itertools import chain

from keras.models import model_from_json
import keras
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Input, Embedding, SpatialDropout1D, Dropout, BatchNormalization, Dense,LSTM, CuDNNGRU
from keras.models import Sequential
from keras.layers import RepeatVector
from keras.layers import TimeDistributed
from keras.utils import plot_model
from keras.models import Model
from keras import optimizers

from tensorflow.python.client import device_lib


from keras import backend as K
K.tensorflow_backend._get_available_gpus()

# Let's set random seed
import tensorflow as tf
tf.set_random_seed(42)
np.random.seed(42)

from sklearn.preprocessing import LabelEncoder, MultiLabelBinarizer, StandardScaler
import itertools
from pprint import pprint

DATA_PATH = ''
%matplotlib inline

# For Google Colab only:
import sys
sys.path.append('/content/gdrive/My Drive/okko competition')
#from reco_utils.recommender.sar.sar_singlenode import SARSingleNode
from google.colab import drive
drive.mount('/content/gdrive')

DATA_PATH = 'gdrive/My Drive/okko competition/data'
CODE_PATH = 'gdrive/My Drive/okko competition/code'

with open(os.path.join(DATA_PATH, 'test_users.json'), 'r') as f:
    test_users = set(json.load(f)['users'])
    
with open(os.path.join(DATA_PATH, 'catalogue.json'), 'r') as f:
    catalogue = json.load(f)
    
catalogue = {int(k): v for k, v in catalogue.items()}
catalogue = pd.DataFrame.from_dict(catalogue,'index').reset_index()
# av_hot - нужно для валидации. Эта колонка равна 1, если фильм или сериал может быть потребленным
catalogue['av_hot'] = catalogue['availability'].apply(lambda x: 1 if x == [] or 'subscription' in x else 0)
catalogue = catalogue.rename(index=str, columns={'index':'element_uid'})
# catalogue.element_uid = catalogue.element_uid.astype('category')
# catalogue['categ_id'] = catalogue.element_uid.cat.codes + 1

Using TensorFlow backend.


Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [0]:
transactions = pd.read_csv(
        os.path.join(DATA_PATH, 'transactions.csv'),
        dtype={
            'element_uid': np.uint16,
            'user_uid': np.uint32,
            'consumption_mode': 'category',
            'ts': np.float64,
            'watched_time': np.uint64,
            'device_type': np.uint8,
            'device_manufacturer': np.uint8
        }
    )

ratings = pd.read_csv(
        os.path.join(DATA_PATH, 'ratings.csv'),
        dtype={
            'element_uid': np.uint16,
            'user_uid': np.uint32,
            'ts': np.float64,
            'rating': np.uint8
        }
    )

bookmarks = pd.read_csv(
        os.path.join(DATA_PATH, 'bookmarks.csv'),
        dtype={
            'element_uid': np.uint16,
            'user_uid': np.uint32,
            'ts': np.float64
        }
    )

bookmarks['rating'] = 5
_bookmarks = bookmarks[['rating', 'user_uid', 'element_uid','ts']]
_ratings = ratings
#_ratings['rating'] = 1

_transaction = transactions[['element_uid', 'user_uid','ts','watched_time','consumption_mode']]
#_transaction = _transaction[(_transaction['watched_time']>700)]
_transaction['rating']=2
_transaction.loc[(_transaction['consumption_mode']=='P')&(_transaction['watched_time']>1000),'rating'] = 8
_transaction.loc[(_transaction['consumption_mode']=='R')&(_transaction['watched_time']>1000),'rating'] = 7
_transaction.loc[(_transaction['consumption_mode']=='S')&(_transaction['watched_time']>1000),'rating'] = 6
_transaction.loc[(_transaction['consumption_mode']=='P')&(_transaction['watched_time']<1000),'rating'] = 5

all_ratings = pd.concat([ _transaction[['user_uid', 'element_uid', 'rating', 'ts']],_bookmarks[['user_uid', 'element_uid', 'rating', 'ts']]])
all_ratings = pd.concat([all_ratings, _ratings[['user_uid', 'element_uid', 'rating','ts']]])
all_ratings = all_ratings.drop_duplicates(['user_uid','element_uid'],keep='first')
all_ratings = all_ratings.sort_values(by=['ts'],ascending=False)
all_ratings= all_ratings.rename(index=str,columns={'element_uid':'itemID','user_uid':'userID','ts':'timestamp'})

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:40: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


In [0]:
t=bookmarks.groupby('element_uid').agg({'ts':'count'}).reset_index()
t['element_uid'] = t['element_uid'].astype('int64')
t = t.rename(index=str, columns={'ts':'count of bookmarks'})
catalogue = catalogue.merge(t,how='left', on='element_uid')

In [0]:
t=all_ratings.groupby('itemID').agg({'rating':'mean'}).reset_index()
t = t.rename(index=str, columns={'rating':'average rating'})
catalogue = catalogue.merge(t,how='left', left_on = 'element_uid', right_on='itemID')

In [0]:
t=all_ratings.groupby('itemID').agg({'rating':'max'}).reset_index()
t = t.rename(index=str, columns={'rating':'max rating'})
catalogue = catalogue.merge(t,how='left', left_on = 'element_uid', right_on='itemID')

In [0]:
t=all_ratings.groupby('itemID').agg({'rating':'min'}).reset_index()
t = t.rename(index=str, columns={'rating':'min rating'})
catalogue = catalogue.merge(t,how='left', left_on = 'element_uid', right_on='itemID')

In [0]:
t=all_ratings.groupby('itemID').agg({'rating':'median'}).reset_index()
t = t.rename(index=str, columns={'rating':'median rating'})
catalogue = catalogue.merge(t,how='left', left_on = 'element_uid', right_on='itemID')

In [0]:
t=all_ratings.groupby('itemID').agg({'rating':'count'}).reset_index()
t = t.rename(index=str, columns={'rating':'count of ratings'})
catalogue = catalogue.merge(t,how='left', left_on = 'element_uid', right_on='itemID')

In [0]:
t = transactions.loc[transactions.consumption_mode == 'S'].groupby('element_uid').agg({'ts':'count'}).reset_index()
t = t.rename(index=str, columns={'ts':'S count of transactions'})
catalogue = catalogue.merge(t,how='left', left_on = 'element_uid', right_on='element_uid')

In [0]:
t = transactions.loc[transactions.consumption_mode == 'P'].groupby('element_uid').agg({'ts':'count'}).reset_index()
t = t.rename(index=str, columns={'ts':'P count of transactions'})
catalogue = catalogue.merge(t,how='left', left_on = 'element_uid', right_on='element_uid')

In [0]:
t = transactions.loc[transactions.consumption_mode == 'R'].groupby('element_uid').agg({'ts':'count'}).reset_index()
t = t.rename(index=str, columns={'ts':'R count of transactions'})
catalogue = catalogue.merge(t,how='left', left_on = 'element_uid', right_on='element_uid')

In [0]:
t = transactions.groupby('element_uid').agg({'watched_time':'sum'}).reset_index()
t = t.rename(index=str, columns={'watched_time': 'number of views'})
catalogue = catalogue.merge(t,how='left', left_on = 'element_uid', right_on='element_uid')
catalogue['number of views'] = catalogue['number of views'] / catalogue['duration']

In [0]:
t = transactions.groupby('element_uid').agg({'watched_time':'mean'}).reset_index()
t = t.rename(index=str, columns={'watched_time': 'mean number of views'})
catalogue = catalogue.merge(t,how='left', left_on = 'element_uid', right_on='element_uid')
catalogue['mean number of views'] = catalogue['mean number of views'] / catalogue['duration']

In [0]:
catalogue = catalogue.fillna(0).replace([np.inf, -np.inf], 0)

### Загрузка данных

Замените `DATA_PATH` на путь к данным, которые вы скачали со страницы соревнования.

In [0]:
all_ratings.itemID = all_ratings.itemID.astype('category')
all_ratings['categ_id'] = all_ratings.itemID.cat.codes + 1
cat_to_element_uid = dict(zip(
        range(1, len(all_ratings.itemID.cat.categories) + 1),
        all_ratings.itemID.cat.categories))

all_ratings.itemID = all_ratings.itemID.astype('category')
all_ratings['categ_id'] = all_ratings.itemID.cat.codes + 1
cat_to_element_uid = dict(zip(
        range(1, len(all_ratings.itemID.cat.categories) + 1),
        all_ratings.itemID.cat.categories
    ))

In [0]:
for attr in [7,10,14,15,16,17,18,19,20,21,25,31,32,42,43,52,54,83,270]:
  

  def func(row,attr=attr):
    if attr in row['attributes']:
      return 1
    else:
      return 0

  catalogue['is'+str(attr)] = catalogue.apply(func,axis=1)
catalogue['ismovie'] = 0
catalogue.loc[catalogue['type']=='movie','ismovie'] = 1
catalogue['ismulti'] = 0
catalogue.loc[catalogue['type']=='multipart_movie','ismulti'] = 1
catalogue['isseries'] = 0
catalogue.loc[catalogue['type']=='series','isseries'] = 1

def av_func_p(row):
  if 'purchase' in row['availability']:
    return 1
  else:
    return 0
def av_func_s(row):
  if 'subscription' in row['availability']:
    return 1
  else:
    return 0
def av_func_r(row):
  if 'rent' in row['availability']:
    return 1
  else:
    return 0
  
def av_func_n(row):
  if len(row['availability']) == 0:
    return 1
  else:
    return 0
catalogue['purch'] = catalogue.apply(av_func_p,axis=1)
catalogue['subs'] = catalogue.apply(av_func_s,axis=1)
catalogue['rent'] = catalogue.apply(av_func_r,axis=1)
catalogue['not_known'] = catalogue.apply(av_func_n,axis=1)


In [0]:
item_features = catalogue[catalogue['element_uid'].isin(all_ratings['itemID'].values)]
item_features = item_features.drop(['type','availability','element_uid','attributes'],axis=1).values
scaler = StandardScaler()
item_features = scaler.fit_transform(item_features)

item_features = np.hstack((np.identity(item_features.shape[0]),item_features))

In [0]:
item_features.shape

(9580, 9629)

In [0]:
from collections import defaultdict

filtered_elements = defaultdict(set)

for user_uid, element_uid in tqdm.tqdm(transactions.loc[:, ['user_uid', 'element_uid']].values):
    if user_uid not in test_users:
        continue
    filtered_elements[user_uid].add(element_uid)

100%|██████████| 9643012/9643012 [02:19<00:00, 69058.82it/s]


---

---

Для примера мы воспользуемся методом K ближайших соседей, реализованным в библиотеке `implicit`. В качестве данных используем только информацию о рейтингах.

Необходимо построить разреженную матрицу, где строкам будут соответствовать элементы, столбцам — пользователи, а на пересечении пользователя и элемента будет находиться количественная оценка степени их взаимодействия, если таковое имело место.

Не забудем добавить `1` к рейтингу, чтобы избежать деления на ноль во время вычисления `tf-idf`.

In [0]:
all_ratings['userID'] = all_ratings['userID'].astype('category')
all_ratings['itemID'] = all_ratings['itemID'].astype('category')

ratings_matrix = sp.coo_matrix(
    (all_ratings['rating'].astype(np.float32) + 1,
        (
            all_ratings['itemID'].cat.codes.copy(),
            all_ratings['userID'].cat.codes.copy()
        )
    )
)

ratings_matrix = ratings_matrix.tocsr()
ratings_matrix_T = ratings_matrix.T.tocsr()

In [0]:
sparsity = ratings_matrix.nnz / (ratings_matrix.shape[0] * ratings_matrix.shape[1])
print('Sparsity: %.6f' % sparsity)
ratings_matrix.shape

Sparsity: 0.002172


(9580, 500000)

Обучить модель крайне просто.

Отображения из оригинальной категории во внутреннюю пригодится нам в дальнейшем.

In [0]:
# основная функция для валидации модели
def validation(dict_):
  d1 = {}
  with open(DATA_PATH+'answer_50.json', 'r') as f:
    d1 = json.load(f)
  av = []


  for key in d1.keys():
      if key in dict_.keys():
          ar1 = np.array(d1[str(key)])
         # print(ar1)
          ar2 = np.array(dict_[str(key)])
          inter = np.intersect1d(ar1,ar2)
          av.append(len(inter))
          to_add = []
          for el in ar1:
              if el  in inter:
                  to_add.append(el)
          for el in ar1:
              if el  not in to_add:
                  to_add.append(el)


          to_add = np.array(to_add)
  return np.mean(av),np.var(av)

def shuffle(dict_):
  d = {}
  d1 = {}
  with open(DATA_PATH+'my_best.json', 'r') as f:
    d1 = json.load(f)
  
  for key in dict_.keys():
    if key in d1.keys():
      ar1 = np.array(dict_[key])
       # print(ar1)
      ar2 = np.array(d1[key])
      inter = np.intersect1d(ar1,ar2)

      to_add = []
      for el in ar1:
        if el  in inter:
          to_add.append(el)
      for el in ar1:
        if el  not in to_add:
          to_add.append(el)
        
        
      to_add = np.array(to_add)
        
      d[key] = list(map(int,list(to_add)))

    else:
      d[key] = dict_[key]
  for key in d1.keys():
    if key not in d.keys():
      d[key] = d1[key]
      
  return d
       


In [0]:
from lightfm import LightFM
from lightfm.evaluation import precision_at_k
from lightfm.evaluation import auc_score
from scipy import sparse
from lightfm.datasets import fetch_movielens

In [0]:
#%%time
model = LightFM(loss='warp',no_components=20, learning_rate=0.01)
#model.fit(ratings_matrix_T, epochs=10, num_threads=4,verbose=True)

model.fit(ratings_matrix_T, item_features=sparse.csr_matrix(item_features), epochs=60, num_threads=4,verbose=True)

Epoch 0
Epoch 1
Epoch 2
Epoch 3
Epoch 4
Epoch 5
Epoch 6
Epoch 7


In [0]:
d1 = {}
t = sparse.csr_matrix(item_features)
for user in tqdm.tqdm(list(test_users)):
   # print(user)
  try:
    cats = list(np.argsort(-model.predict(user_uid_to_cat[int(user)],[i for i in element_uid_to_cat.values()],\
                                           item_features=t,
                                          num_threads=30  )))[:80]

    d1[str(user)] = [int(all_ratings['itemID'].cat.categories[i]) for i in cats if i not in filtered_elements_cat[user]]
      #print(d[str(user)])
    d1[str(user)] = d1[str(user)][:20]
  except:
    pass
print(validation(d1))

to_send = shuffle(d1)
    


In [0]:
with open(os.path.join(DATA_PATH, 'answer_40_epochs_lightfm_with_features.json'), 'w') as f:
    json.dump(to_send, f)

In [0]:
np.argsort(-model.predict(user_uid_to_cat[13],[i for i in element_uid_to_cat.values()],\
                                           item_features=t,num_threads=30  ))